# Article CMS - Cloud Developer With MS Azure - Udacity Project I

## 1. Add configuration

In [ ]:
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('azure.cfg'))

RESOURCE_GROUP              = config.get('RESOURCE','RESOURCE_GROUP')
LOCATION                    = config.get('RESOURCE','LOCATION')
SQL_SERVER                  = config.get("SQL","SQL_SERVER")
SQL_DATABASE                = config.get("SQL","SQL_DATABASE")
SQL_ADMIN_USER              = config.get("SQL","SQL_ADMIN_USER")
SQL_ADMIN_PASSWORD          = config.get("SQL","SQL_ADMIN_PASSWORD")
SQL_SKU                     = config.get("SQL","SQL_SKU")
SQL_EDITION                 = config.get("SQL","SQL_EDITION")
STORAGE_ACCOUNT_NAME        = config.get("STORAGE","STORAGE_ACCOUNT_NAME")
STORAGE_CONTAINER_NAME      = config.get("STORAGE","STORAGE_CONTAINER_NAME")
STORAGE_SKU                 = config.get("STORAGE","STORAGE_SKU")
STORAGE_KIND                = config.get("STORAGE","STORAGE_KIND")
APP_NAME                    = config.get("APP","APP_NAME")
APP_SERVICE_PLAN_NAME       = config.get("APP","APP_SERVICE_PLAN_NAME")
APP_SKU                     = config.get("APP","APP_SKU")
APP_RUNTIME                 = config.get("APP","APP_RUNTIME")
AAD_APP_NAME                = config.get("AAD","AAD_APP_NAME")
REDIRECT_URL                = config.get("AAD","REDIRECT_URL")
PUBLIC_IP_ADDRESS           = config.get("IP","PUBLIC_IP_ADDRESS")

In [ ]:
import os
os.environ["RESOURCE_GROUP"]            = RESOURCE_GROUP
os.environ["LOCATION"]                  = LOCATION
os.environ["SQL_SERVER"]                = SQL_SERVER
os.environ["SQL_DATABASE"]              = SQL_DATABASE
os.environ["SQL_ADMIN_USER"]            = SQL_ADMIN_USER
os.environ["SQL_ADMIN_PASSWORD"]        = SQL_ADMIN_PASSWORD
os.environ["SQL_SKU"]                   = SQL_SKU
os.environ["SQL_EDITION"]               = SQL_EDITION
os.environ["STORAGE_ACCOUNT_NAME"]      = STORAGE_ACCOUNT_NAME
os.environ["STORAGE_CONTAINER_NAME"]    = STORAGE_CONTAINER_NAME
os.environ["STORAGE_SKU"]               = STORAGE_SKU
os.environ["STORAGE_KIND"]              = STORAGE_KIND
os.environ["APP_NAME"]                  = APP_NAME
os.environ["APP_SERVICE_PLAN_NAME"]     = APP_SERVICE_PLAN_NAME
os.environ["APP_SKU"]                   = APP_SKU
os.environ["APP_RUNTIME"]               = APP_RUNTIME
os.environ["AAD_APP_NAME"]              = AAD_APP_NAME
os.environ["REDIRECT_URL"]              = REDIRECT_URL
os.environ["PUBLIC_IP_ADDRESS"]         = PUBLIC_IP_ADDRESS

## 2. Create Resources

### 2.1. Login Azure CLI

In [ ]:
! az login

### 2.2. Create Resource Group

#### 2.2.1. See list-locations

In [ ]:
! az account list-locations -o table

#### 2.2.2. Create resource group

In [ ]:
! az account list-locations -o table

#### 2.2.2. Create resource group

In [ ]:
! az group create \
    --name {RESOURCE_GROUP} \
    --location {LOCATION}

### 2.3 Create SQL Database

#### 2.3.1. Create SQL Server

In [ ]:
! az sql server create \
    -n {SQL_SERVER} \
    -g {RESOURCE_GROUP} \
    -l {LOCATION} \
    -u {SQL_ADMIN_USER} \
    -p {SQL_ADMIN_PASSWORD} \
    -e True

#### 2.3.2. Create SQL Database

In [ ]:
! az sql db create \
    -n {SQL_DATABASE} \
    -g {RESOURCE_GROUP} \
    -s {SQL_SERVER} \
    -e {SQL_EDITION} \
    --bsr Local \
    -z False

#### 2.3.3. Config firewall rules connect from azure services

In [ ]:
! az sql server firewall-rule create \
    -g {RESOURCE_GROUP} \
    -s {SQL_SERVER} \
    -n azureaccess \
    --start-ip-address 0.0.0.0 \
    --end-ip-address 0.0.0.0 \
    --verbose

#### 2.3.4. Config firewall rules connect from local machine

In [ ]:
! az sql server firewall-rule create \
    -g {RESOURCE_GROUP} \
    -s {SQL_SERVER} \
    -n clientip \
    --start-ip-address {PUBLIC_IP_ADDRESS} \
    --end-ip-address {PUBLIC_IP_ADDRESS} \
    --verbose

### 2.4. Create Azure SQL Database tables: Posts and Users - If using Windows

#### 2.4.1. Config pyodbc

In [ ]:
import pyodbc

In [ ]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};' +
    'SERVER=' + {SQL_SERVER} + ';' +
    'DATABASE=' + {SQL_DATABASE} + ';' +
    'UID=' + {SQL_ADMIN_USER} + ';' +
    'PWD=' + {SQL_ADMIN_PASSWORD} + ';' +
    'Encrypt=yes;' +
    'TrustServerCertificate=no;' +
    'Connection Timeout=30;'
)

In [ ]:
cursor = conn.cursor()

#### 2.4.2. Create Posts Table

In [ ]:
sql_script = ''
# Read the SQL script file
with open('./sql_scripts/posts-table-init.sql', 'r') as file:
    sql_script = file.read()

# Execute the SQL script
cursor.execute(sql_script)

# Commit the transaction
conn.commit()

#### 2.4.3. Create Users Table

In [ ]:
sql_script = ''
# Read the SQL script file
with open('./sql_scripts/users-table-init.sql', 'r') as file:
    sql_script = file.read()

# Execute the SQL script
cursor.execute(sql_script)

# Commit the transaction
conn.commit()

#### 2.4.4. Check tables

In [ ]:
sql_script = 'SELECT * FROM posts;'

# Execute the SQL script
cursor.execute(sql_script)

# Commit the transaction
conn.commit()

In [ ]:
sql_script = 'SELECT * FROM users;'

# Execute the SQL script
cursor.execute(sql_script)

# Commit the transaction
conn.commit()

### 2.5 Create Storage Container

#### 2.5.1. Create Azure Storage Account

In [ ]:
! az storage account create \
    --name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {STORAGE_SKU} \
    --kind {STORAGE_KIND}


#### 2.5.2. Show Azure Storage Account Connection String

In [ ]:
! az storage account show-connection-string \
    --name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP}

#### 2.5.3 Create Storage Container - Get the above connection string and update and rung the following commands

In [ ]:
! az storage container create \
    --name {STORAGE_CONTAINER_NAME} \
    --account-name {STORAGE_ACCOUNT_NAME} \
    --auth-mode login \
    --public-access container

In [ ]:
! az storage account keys list \
    --account-name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --output json


- Update key in the result of the above command to azure.cfg file

### 2.6 Create Azure App Service

#### 2.6.1. List Runtime Environment

In [ ]:
! az webapp list-runtimes

#### 2.6.2. Create App Service Plan

In [ ]:
#### 2.6.2. Create App Service Plan

In [ ]:
! az appservice plan create \
    --name {APP_SERVICE_PLAN_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {APP_SKU} \
    --is-linux

#### 2.6.3. Create App Service

In [ ]:
#### 2.6.3. Create App Service

In [ ]:
! az webapp create \
    --name {APP_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --plan {APP_SERVICE_PLAN_NAME} \
    --runtime {APP_RUNTIME}

### 3. Config Azure Active Directory

#### 3.1. Register an application

In [ ]:
! az ad app create \
    --display-name congdinh2025-article-cms \
    --web-redirect-uris {REDIRECT_URL} \
    --sign-in-audience AzureADandPersonalMicrosoftAccount


#### 3.2. Config client secret for application
- After registering an application
- Create a credential for the application
- Using id application from result of the above command

In [ ]:
! az ad app credential reset \
    --id d7c0c4ea-6eb8-46c5-8593-2916a5f08c6a \
    --query "password" -o json


- Update secret from the result of the above command to azure.cfg file

## 4. Deploy Application

### 4.1. Using VSCode - Recommended

- Login Azure by Azure Extension
- Right click on App Service Name the app service in Azure Resources Manager - The Azure Extension tab. 
- Choose Deploy to Web App

### 4.2. Using Azure CLI

#### 4.2.1 New Deploy

In [ ]:
! az webapp up \
    --resource-group {RESOURCE_GROUP} \
    --name {APP_NAME} \
    --sku {APP_SKU} \
    --verbose


#### 4.2.2. Update After Deployment

In [ ]:
! az webapp up \
    --name {APP_NAME} \
    --verbose

### 5. Delete Resources

#### 5.1. Delete group - delete all resources belong to this group

In [ ]:
! az group delete -n {RESOURCE_GROUP}

### 5.2. Delete each resources

#### 5.2.1. Delete Azure Sql Database

In [ ]:
! az sql server delete \
    --name {SQL_SERVER} \
    --resource-group {RESOURCE_GROUP} \

In [ ]:
! az sql db delete \
    --name {SQL_DATABASE} \
    --resource-group {RESOURCE_GROUP} \
    --server {SQL_SERVER}

#### 5.2.2. Delete Azure Storage Container

In [ ]:
! az storage container delete \
    --name {STORAGE_CONTAINER_NAME} \
    --account-name {STORAGE_ACCOUNT_NAME} 

In [ ]:
! az storage account delete \
    --name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP}